In [1]:
import pickle
import numpy as np
with open('train.pkl', 'rb') as file:
    data = pickle.load(file)

In [2]:
train=data["train_data"]
train[0]

array([[2.0120000e+03, 1.0000000e+00, 1.0000000e+00, ..., 9.2000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [2.0120000e+03, 1.0000000e+00, 1.0000000e+00, ..., 1.5100000e+01,
        0.0000000e+00, 0.0000000e+00],
       [2.0120000e+03, 1.0000000e+00, 1.0000000e+00, ..., 1.8317552e+01,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [2.0120000e+03, 1.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [2.0120000e+03, 1.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [2.0120000e+03, 1.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]])

In [3]:
train=data["train_data"]
train_data_expanded = np.expand_dims(train, axis=-1)
train_data_expanded.shape

(8784, 63, 18, 1)

In [4]:
train_data_expanded = np.transpose(train_data_expanded, (0, 2, 1,3))
train_data_expanded.shape

(8784, 18, 63, 1)

In [5]:
train_data=train_data_expanded[:,5:,:,:]

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ModifiedAFFWithSoftmax(nn.Module):
    def __init__(self, channels=13, r=4, num_features=13):
        super(ModifiedAFFWithSoftmax, self).__init__()
        self.num_features = num_features
        inter_channels = int(channels // r * num_features)
        
        self.local_att = nn.Sequential(
            nn.Conv2d(channels, inter_channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(inter_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(inter_channels, channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(channels),
        )
        
        self.global_att = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, inter_channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(inter_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(inter_channels, channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(channels),
        )

    def forward(self, x):
        xl = self.local_att(x)
        xg = self.global_att(x)
        xlg = xl + xg
        
        weights = xlg.view(xlg.size(0), self.num_features, -1)
        weights = F.softmax(weights, dim=1)
        weights = weights.view_as(xlg)
        
        separated_weights = torch.chunk(weights, self.num_features, dim=1)
        weighted_features = [x[:, i:i+1, :, :] * separated_weights[i] for i in range(self.num_features)]
        
        output = torch.sum(torch.stack(weighted_features, dim=0), dim=0)

        return output.squeeze(1)

# 初始化模型
model = ModifiedAFFWithSoftmax()

# 生成模拟输入数据
train_tensor = torch.from_numpy(train_data)  # Batch size 8784, 18 features, 41 sites, width 1
train_tensor = train_tensor.float()

# 前向传播
output_tensor = model(train_tensor)

# 打印输出形状
print("Output shape:", output_tensor.shape)

d:\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Output shape: torch.Size([8784, 63, 1])
